In [9]:
from torch import nn
import torch
from torchsummary import summary
from torch import Tensor

<img src = 'https://i.imgur.com/Zm976b9.png'>

* 논문: https://arxiv.org/pdf/1512.03385

## [1] BasicBlock

In [10]:
class BasicBlock(nn.Module):
    expansion_factor = 1
    def __init__(self, in_channels:int, out_channels:int, stride:int = 1):
        super().__init__()
            
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias = False),
            nn.BatchNorm2d(out_channels)
        )
        if stride != 1 or in_channels != out_channels * self.expansion_factor:
            self.residual = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion_factor, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(out_channels * self.expansion_factor)
            )
        else:
            self.residual = nn.Sequential()
        
        self.relu = nn.ReLU(inplace = True)
    
    def forward(self, x:Tensor) -> Tensor:
        resid = x
        out = self.block(x)
        out += self.residual(resid)
        out = self.relu(out)
        return out

In [5]:
model = BasicBlock(3,64)
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,728
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,864
       BatchNorm2d-5         [-1, 64, 224, 224]             128
            Conv2d-6         [-1, 64, 224, 224]             192
       BatchNorm2d-7         [-1, 64, 224, 224]             128
              ReLU-8         [-1, 64, 224, 224]               0
Total params: 39,168
Trainable params: 39,168
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 196.00
Params size (MB): 0.15
Estimated Total Size (MB): 196.72
----------------------------------------------------------------


## [2] BottleNeck

In [11]:
class BottleNeck(nn.Module):
    expansion_factor = 4
    def __init__(self, in_channels:int, out_channels:int, stride:int = 1):
        super().__init__()
        
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(out_channels, out_channels * self.expansion_factor, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm2d(out_channels * self.expansion_factor)
        )
        
        if stride != 1 or in_channels != out_channels * self.expansion_factor:
            self.residual = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion_factor, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(out_channels * self.expansion_factor)
            )
        else:
            self.residual = nn.Sequential()
        
        self.relu = nn.ReLU(inplace = True)
            
    def forward(self, x:Tensor) -> Tensor:
        resid = x
        out = self.block(x)
        out += self.residual(resid)
        out = self.relu(out)
        return out

In [7]:
model = BottleNeck(3,64)
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]             192
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,864
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
            Conv2d-7        [-1, 256, 224, 224]          16,384
       BatchNorm2d-8        [-1, 256, 224, 224]             512
            Conv2d-9        [-1, 256, 224, 224]             768
      BatchNorm2d-10        [-1, 256, 224, 224]             512
             ReLU-11        [-1, 256, 224, 224]               0
Total params: 55,488
Trainable params: 55,488
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/ba

## [3] ResNet

In [15]:
class ResNet_blueprint(nn.Module):
    def __init__(self, block, num_blocks, num_classes):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        )
        
        self.conv2 = self._make_layer(block, 64, num_blocks[0], stride = 1)
        self.conv3 = self._make_layer(block, 128, num_blocks[1], stride = 2)
        self.conv4 = self._make_layer(block, 256, num_blocks[2], stride = 2)
        self.conv5 = self._make_layer(block, 512, num_blocks[3], stride = 2)
        
        self.avgpool = nn.AdaptiveAvgPool2d(output_size = (1,1))
        self.fc = nn.Linear(512 * block.expansion_factor, num_classes)
        self._init_layer()
    
    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion_factor
        return nn.Sequential(*layers)
    
    def _init_layer(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity = 'relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x:Tensor) -> Tensor:
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        
        out = self.avgpool(out)
        out = torch.flatten(out,1)
        out = self.fc(out)
        return out

In [16]:
class ResNet:
    def __init__(self, num_classes):
        self.num_classes = num_classes
        
    def ResNet18(self):
        return ResNet_blueprint(BasicBlock, [2,2,2,2], self.num_classes)
    
    def ResNet34(self):
        return ResNet_blueprint(BasicBlock, [3,4,6,3], self.num_classes)
        
    def ResNet50(self):
        return ResNet_blueprint(BottleNeck, [3,4,6,3], self.num_classes)
        
    def ResNet101(self):
        return ResNet_blueprint(BottleNeck, [3,4,23,3], self.num_classes)
    
    def ResNet152(self):
        return ResNet_blueprint(BottleNeck, [3,8,36,3], self.num_classes)

In [17]:
model = ResNet(10).ResNet18()
summary(model, (3,32,32), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6

In [18]:
model = ResNet(10).ResNet152()
summary(model, (3,32,32), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,096
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          16,384
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13            [-1, 256, 8, 8]          16,384
      BatchNorm2d-14            [-1, 25

## [5] 요약

In [2]:
import torch
from torch import nn
from torchsummary import summary

In [3]:
class Basicblock(nn.Module):
    expansion_factor = 1
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias = False),
            nn.BatchNorm2d(out_channels)
        )
        
        if stride != 1 or in_channels != out_channels * self.expansion_factor:
            self.residual = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion_factor, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(out_channels * self.expansion_factor)
            )
        else:
            self.residual = nn.Sequential()
        
        self.relu = nn.ReLU(inplace = True)
    
    def forward(self, x):
        resid = x
        out = self.block(x)
        out += self.residual(resid)
        out = self.relu(out)
        return out

model = Basicblock(3,64,1)
summary(model, (3,32,32),device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
            Conv2d-6           [-1, 64, 32, 32]             192
       BatchNorm2d-7           [-1, 64, 32, 32]             128
              ReLU-8           [-1, 64, 32, 32]               0
Total params: 39,168
Trainable params: 39,168
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 4.00
Params size (MB): 0.15
Estimated Total Size (MB): 4.16
----------------------------------------------------------------


In [4]:
class BottleNeckblock(nn.Module):
    expansion_factor = 4
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = stride, padding = 1, bias = False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(out_channels, out_channels * self.expansion_factor, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm2d(out_channels * self.expansion_factor),
        )
        
        if stride != 1 or in_channels != out_channels * self.expansion_factor:
            self.residual = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion_factor, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(out_channels * self.expansion_factor)
            )
        else:
            self.residual = nn.Sequential()
            
        self.relu = nn.ReLU(inplace = True)
        
    def forward(self, x):
        resid = x
        out = self.block(x)
        out += self.residual(resid)
        out = self.relu(out)
        return out

model = BottleNeckblock(3,64,1)
summary(model, (3,32,32),device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]             192
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
            Conv2d-7          [-1, 256, 32, 32]          16,384
       BatchNorm2d-8          [-1, 256, 32, 32]             512
            Conv2d-9          [-1, 256, 32, 32]             768
      BatchNorm2d-10          [-1, 256, 32, 32]             512
             ReLU-11          [-1, 256, 32, 32]               0
Total params: 55,488
Trainable params: 55,488
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/ba

In [5]:
class ResNet_blueprint(nn.Module):
    def __init__(self, block, num_blocks, num_classes):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        )
        self.layer1 = self._make_layer(block, 64, num_blocks[0], 1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], 2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], 2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], 2)
        
        self.avg = nn.AdaptiveAvgPool2d(output_size = (1,1))
        
        self.fc = nn.Linear(512 * block.expansion_factor, num_classes)
        
        self._init_layer()
        
    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks-1)  
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion_factor
        return nn.Sequential(*layers)

    def _init_layer(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity = 'relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg(out)
        out = torch.flatten(out,1)
        out = self.fc(out)
        return out

model = ResNet_blueprint(BottleNeckblock, [3, 8, 36, 3], 10)
summary(model, (3,32,32), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,096
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          16,384
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13            [-1, 256, 8, 8]          16,384
      BatchNorm2d-14            [-1, 25

In [9]:
class ResNet:
    def __init__(self, num_classes):
        self.num_classes = num_classes
    
    def ResNet18(self):
        return ResNet_blueprint(Basicblock, [2,2,2,2], self.num_classes)
    
    def ResNet34(self):
        return ResNet_blueprint(Basicblock, [3,4,6,3], self.num_classes)
        
    def ResNet50(self):
        return ResNet_blueprint(BottleNeckblock, [3,4,6,3], self.num_classes)
        
    def ResNet101(self):
        return ResNet_blueprint(BottleNeckblock, [3,4,23,3], self.num_classes)
        
    def ResNet152(self):
        return ResNet_blueprint(BottleNeckblock, [3,8,36,3], self.num_classes)

model = ResNet(1000).ResNet152()
summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [7]:
from torchvision.models import resnet152
model = resnet152()

summary(model, (3,224,224), device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

## cifar10사용예시

In [ ]:
def training(self, model, data_loader, criterion, optimizer, device, batch_size):
    model.train()
    train_loss = 0.
    train_acc = 0.
    with tqdm(data_loader, unit = 'batch') as tepoch:
        for i, (X, y) in enumerate(tepoch):
            tepoch.set_description('Training')
            X = X.to(device)
            y = y.to(device)
            
            y_hat = model(X)
            loss = criterion(y_hat, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            iter_loss = loss.item()
            train_loss += iter_loss
            
            pred = y_hat.max(1, keepdim = True)[1]
            iter_acc = pred.eq(y.data.view_as(pred)).sum().item()
            train_acc += iter_acc
            
            tepoch.set_postfix(iteration_num = f'[{i}/{len(data_loader)}]',
                              train_iter_loss = f'{iter_loss / batch_size:.3f}',
                              train_iter_accuracy = f'{iter_acc / batch_size*100:.2f}%')
            
    train_loss = train_loss / len(data_loader)
    train_acc = train_acc / len(data_loader)
    return train_loss, train_acc

def evaluation(self, model, data_loader, criterion, device, batch_size):
    model.eval()
    valid_loss = 0.
    valid_acc = 0.
    with torch.no_grad():
        with tqdm(data_loader, unit = 'batch') as tepoch:
            for i, (X, y) in enumerate(tepoch):
                tepoch.set_description('Evaluation')
                X = X.to(device)
                y = y.to(device)
                
                y_hat = model(X)
                loss = criterion(y_hat, y)
                
                iter_loss = loss.item()
                valid_loss += iter_loss
                
                pred = y_hat.max(1, keepdim = True)[1]
                iter_acc = pred.eq(y.data.view_as(pred)).sum().item()
                valid_acc += iter_acc
                
                tepoch.set_postfix(iteration_num = f'[{i}/{len(data_loader)}]',
                                  valid_iter_loss = f'{iter_loss / batch_size:.3f}',
                                  valid_iter_accuracy = f'{iter_acc / batch_size*100:.2f}%')
                
    valid_loss = valid_loss / len(data_loader)
    valid_acc = valid_acc / len(data_loader)
    return valid_loss, valid_acc